In [ ]:
import cv2
import imutils
import numpy as np
from subprocess import call
import time
import os
import glob
import smtplib
import base64
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
#from email.MIMEText import MIMEText
from email.mime.text import MIMEText
import sys
import socket
from playsound import playsound
import serial

# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))
port=serial.Serial("COM3",baudrate=9600, timeout=.3)

# Loading image
cap = cv2.VideoCapture(0)
def mail():

    gmail_user = "iotdemopan@gmail.com"
    gmail_pwd = "paniotdemo987@"
    FROM = 'iotdemopan@gmail.com'
    TO = ['jawaharlakshmanan6@gmail.com'] #must be a list

    #IMAGE

    msg = MIMEMultipart()
    time.sleep(1)
    msg['Subject'] ="Alert...."

    #BODY with 2 argument

    #body=sys.argv[1]+sys.argv[2]
##    body="hello"          
##    msg.attach(MIMEText(body,'plain'))
##    time.sleep(1)


    #IMAGE
    fp = open("1.jpg", 'rb')  
    time.sleep(1)
    img = MIMEImage(fp.read())
    time.sleep(1)
    fp.close()
    time.sleep(1)
    msg.attach(img)
    time.sleep(1)

    try:
            server = smtplib.SMTP("smtp.gmail.com", 587) #or port 465 doesn't seem to work!
            print ("smtp.gmail")
            server.ehlo()
            print ("ehlo")
            server.starttls()
            print ("starttls")
            server.login(gmail_user, gmail_pwd)
            print ("reading mail & password")
            server.sendmail(FROM, TO, msg.as_string())
            print ("from")
            server.close()
            print ('successfully sent the mail')
    except:
            print ("failed to send mail")

while True:
   
    ret,img=cap.read()
    img = imutils.resize(img, width=500)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            print(label)
            time.sleep(2)
            if(label == 'cat'or'dog'or'horse'or'sheep'or'cow'or'elephant'or'bear'or'zebra'or'giraffe'):
                cv2.imwrite("1.jpg",img)
                playsound('aud2.mpeg')
                mail()
                port.write(str.encode('A'))
            else:
                port.write(str.encode('C'))

            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
           

    # show the output frame
    cv2.imshow("Frame", img)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
            break
cv2.destroyAllWindows()



In [17]:
import cv2
#import imutils
import numpy as np
from subprocess import call
import time
import os
import glob
import smtplib
import base64
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
#from email.MIMEText import MIMEText
from email.mime.text import MIMEText
import sys
import socket
from playsound import playsound
import serial

## ------------------------- tensorflow ------------------------------
#%%bash
#cd models/research/
!"C:\Users\jawah\OneDrive\Desktop\tensorflow\protoc-3.15.3-win64\bin\protoc.exe" object_detection/protos/*.proto --python_out=.
# 
# %%bash 
# cd models/research
# pip install .


import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model


# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

import pathlib

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
## TEST_IMAGE_PATHS

model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

print(detection_model.signatures['serving_default'].inputs)
detection_model.signatures['serving_default'].output_dtypes

detection_model.signatures['serving_default'].output_shapes


def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict



def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = image_path #np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  try:
    image_np,classed = vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          line_thickness=8)
    return image_np, classed

  except:
        image_np = vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          line_thickness=8)
        return image_np
    


  #display(Image.fromarray(image_np))
  #return image_np, classed



## changes 1

#port=serial.Serial("COM3",baudrate=9600, timeout=.3)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [27]:

# Loading image
cap = cv2.VideoCapture(0)
def mail():
    gmail_user = "iotdemopan@gmail.com"
    gmail_pwd = "paniotdemo987@"
    FROM = 'iotdemopan@gmail.com'
    TO = ['jawaharlakshmanan6@gmail.com'] #must be a list
    #IMAGE
    msg = MIMEMultipart()
    time.sleep(1)
    msg["From"] = FROM
    msg["To"] = ",".join(TO)
    msg['Subject'] ="Alert...."
    #BODY with 2 argument

    #body=sys.argv[1]+sys.argv[2]
##    body="hello"
##    msg.attach(MIMEText(body,'plain'))
##    time.sleep(1)
    #IMAGE
    fp = open("1.jpg", 'rb')
    time.sleep(1)
    img = MIMEImage(fp.read())
    time.sleep(1)
    fp.close()
    time.sleep(1)
    msg.attach(img)
    time.sleep(1)


    server = smtplib.SMTP("smtp.gmail.com", 587) #or port 465 doesn't seem to work!
    print ("smtp.gmail")
    server.ehlo()
    print ("ehlo")
    server.starttls()
    print ("starttls")
    server.ehlo()
    print ("ehlo")
    server.login(gmail_user, gmail_pwd)
    print ("reading mail & password")
    server.sendmail(FROM, TO, msg.as_string())
    print ("from")
    server.quit()
    print ('successfully sent the mail')

mail()

smtp.gmail
ehlo
starttls


SMTPServerDisconnected: Connection unexpectedly closed

In [20]:
## <--------------------------------- opencv ---------------------->
## FOR VIDEO CAMERA

import cv2

vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 10000)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 18000)
vid.set(cv2.CAP_PROP_FPS, 25)

while True:

    ret,frame =cap.read()
    try :
        frame, cls = show_inference(detection_model, frame)
        ## changes 2
        ## changes 3
        # DISPLAYING THE BOUND OF PROPABALILITY
        elephant_count = 0
        for i in cls:
            if(i.split()[0][:-1] == "elephant"):#'cat'or'dog'or'horse'or'sheep'or'cow'or'elephant'or'bear'or'zebra'or'giraffe'):
                if float(i.split()[1][:-1]) >= 75.0:
                    print("we found elephant please call the forest ranger")
                    print("elephant count : ", cls)
                    cv2.imwrite("1.jpg", frame)
        #            playsound('aud2.mpeg')
                    mail()
         #           port.write(str.encode('A'))
                
                else:
                    print("something found like an elephant")
          #          port.write(str.encode('C'))
    except:
        frame = show_inference(detection_model, frame)

    # Display the resulting frame 
    cv2.imshow("frame", frame) 
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows()     





something found like an elephant
something found like an elephant
something found like an elephant
something found like an elephant
something found like an elephant
something found like an elephant
something found like an elephant
we found elephant please call the forest ranger
elephant count :  ['elephant: 84.0%']
smtp.gmail
ehlo
starttls
failed to send mail


In [32]:
def mail():
    gmail_user = "jawaharlakshmanan6@gmail.com" #"iotdemopan@gmail.com"
    gmail_pwd = "BorntowiN"#"paniotdemo987"
    FROM = "jawaharlakshmanan6@gmail.com"#'iotdemopan@gmail.com'
    TO = ['jawaharlakshmanan6@gmail.com',"jaiaravindh111399@gmail.com "] #must be a list
    #IMAGE
    msg = MIMEMultipart()
    time.sleep(1)
    msg["From"] = FROM
    msg["To"] = TO
    msg['Subject'] ="IOT message Alert...."
    #BODY with 2 argument

    #body=sys.argv[1]+sys.argv[2]
##    body="hello"
##    msg.attach(MIMEText(body,'plain'))
##    time.sleep(1)
    #IMAGE
    fp = open("1.jpg", 'rb')
    time.sleep(1)
    img = MIMEImage(fp.read())
    time.sleep(1)
    fp.close()
    time.sleep(1)
    msg.attach(img)
    time.sleep(1)


    server = smtplib.SMTP("smtp.gmail.com", 587) #or port 465 doesn't seem to work!
    print ("smtp.gmail")
    server.ehlo()
    print ("ehlo")
    server.starttls()
    print ("starttls")
    server.ehlo()
    print ("ehlo")
    server.login(gmail_user, gmail_pwd)
    print ("reading mail & password")
    server.sendmail(FROM, TO, msg.as_string())
    print ("from")
    server.quit()
    print ('successfully sent the mail')
mail()

smtp.gmail
ehlo
starttls
ehlo
reading mail & password


AttributeError: 'list' object has no attribute 'encode'

In [40]:
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart


def SendMail():
    img_data = open("1.jpg", 'rb').read()
    msg = MIMEMultipart()
    From = 'jawaharlakshmanan6@gmail.com'
    To = ["jaiaravindh111399@gmail.com","thomassjerin3@gmail.com"]
    msg['Subject'] = 'subject : nothing'
    msg['From'] = 'jawaharlakshmanan6@gmail.com'
    msg['To'] = "thomassjerin3@gmail.com "

    text = MIMEText("test")
    msg.attach(text)
    image = MIMEImage(img_data, name=os.path.basename("1.jpg"))
    msg.attach(image)

    s = smtplib.SMTP("smtp.gmail.com", 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login("jawaharlakshmanan6@gmail.com", "BorntowiN")
    s.sendmail(From, To, msg.as_string())
    s.quit()

In [41]:
SendMail()

In [3]:
import cv2

In [29]:

import cv2

class VideoCamera:
    def __init__(self):
        self.vid = cv2.VideoCapture(0)
        self.vid.set(cv2.CAP_PROP_FRAME_WIDTH, 10000)
        self.vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 18000)
        self.vid.set(cv2.CAP_PROP_FPS, 25)
        while True:
            self.ret, self.frame = self.vid.read()
            if self.ret:
                cv2.imshow("fds", self.frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        self.vid.release()
        # Destroy all the windows
        cv2.destroyAllWindows()

        



    def frame_capture(self):
        pass

In [30]:
v = VideoCamera()

In [9]:
v.get_frame()

In [45]:
import smtplib
import base64
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
#from email.MIMEText import MIMEText
from email.mime.text import MIMEText
import os

In [49]:


class Mail:
    def __init__(self):
        self.User_Detail()
        self.Receiver_detail()

    def User_Detail(self):
        self.From = "jawaharlakshmanan6@gmail.com"
        self.password = "BorntowiN"

    def Receiver_detail(self):
        self.To = ["jaiaravindh111399@gmail.com","thomassjerin3@gmail.com"]
        self.to = "jaiaravindh111399@gmail.com"
    
    def read_image(self):
        self.file_name = "1.jpg"
        self.image_data = open(self.file_name,"rb").read()
        return self.image_data

    def Message_Creation(self):
        msg = MIMEMultipart()
        msg["From"] = self.From
        msg["To"] = self.to
        msg["subject"] =  'subject : Alert!! '+' Animal found...'
        text = MIMEText("test")
        return text, msg

    def Message_Passing(self):
        text, msg = self.Message_Creation()
        msg.attach(text)
        # image getting from read_image
        img_data = self.read_image()
        image = MIMEImage(img_data, name=os.path.basename(self.file_name))
        msg.attach(image)
        s = smtplib.SMTP("smtp.gmail.com", 587)
        s.ehlo()
        s.starttls()
        s.ehlo()
        s.login(self.From, self.password)
        s.sendmail(self.From, self.To, msg.as_string())
        s.quit()
        print ('successfully sent the mail')
    

In [50]:
m1 = Mail()

In [51]:
m1.Message_Passing()

successfully sent the mail


In [52]:

## tf

In [53]:

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util



In [64]:
import pathlib


class Tensorflow_mobile_SSD:
    def __init__(self):
        # patch tf1 into `utils.
        utils_ops.tf = tf.compat.v1
        # Patch the location of gfile
        tf.gfile = tf.io.gfile
        # List of the strings that is used to add correct label for each box.
        self.PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
        self.category_index = label_map_util.create_category_index_from_labelmap(self.PATH_TO_LABELS, use_display_name=True)
        
        # If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
        self.PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
        self.TEST_IMAGE_PATHS = sorted(list(self.PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
        ## TEST_IMAGE_PATHS
        
        model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
        self.detection_model = self.load_model(model_name)
        print(self.detection_model.signatures['serving_default'].inputs)
        self.detection_model.signatures['serving_default'].output_dtypes
        self.detection_model.signatures['serving_default'].output_shapes

        

    def load_model(self, model_name):
      self.base_url = 'http://download.tensorflow.org/models/object_detection/'
      self.model_file = model_name + '.tar.gz'
      self.model_dir = tf.keras.utils.get_file(
        fname=model_name,
        origin=self.base_url + self.model_file,
        untar=True)
    
      self.model_dir = pathlib.Path(self.model_dir)/"saved_model"
    
      self.model = tf.saved_model.load(str(self.model_dir))
    
      return self.model
    
    def run_inference_for_single_image(self, model, image):
      self.image = np.asarray(image)
      # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
      self.input_tensor = tf.convert_to_tensor(self.image)
      # The model expects a batch of images, so add an axis with `tf.newaxis`.
      self.input_tensor = self.input_tensor[tf.newaxis,...]
    
      # Run inference
      self.model_fn = model.signatures['serving_default']
      self.output_dict = self.model_fn(self.input_tensor)
    
      # All outputs are batches tensors.
      # Convert to numpy arrays, and take index [0] to remove the batch dimension.
      # We're only interested in the first num_detections.
      self.num_detections = int(self.output_dict.pop('num_detections'))
      self.output_dict = {key:value[0, :self.num_detections].numpy()
                     for key,value in self.output_dict.items()}
      self.output_dict['num_detections'] = self.num_detections
    
      # detection_classes should be ints.
      self.output_dict['detection_classes'] = self.output_dict['detection_classes'].astype(np.int64)
    
      # Handle models with masks:
      if 'detection_masks' in self.output_dict:
        # Reframe the the bbox mask to the image size.
        self.detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  self.output_dict['detection_masks'], self.output_dict['detection_boxes'],
                   self.image.shape[0], self.image.shape[1])
        self.detection_masks_reframed = tf.cast(self.detection_masks_reframed > 0.5,
                                           tf.uint8)
        self.output_dict['detection_masks_reframed'] = self.detection_masks_reframed.numpy()
    
      return self.output_dict
    
    def show_inference(self, model, image_path):
       # the array based representation of the image will be used later in order to prepare the
       # result image with boxes and labels on it.
       self.image_np = image_path #np.array(Image.open(image_path))
       # Actual detection.
       self.output_dict = self.run_inference_for_single_image(model, self.image_np)
       # Visualization of the results of a detection.
       try:
         self.image_np,self.classed = vis_util.visualize_boxes_and_labels_on_image_array(
               self.image_np,
               self.output_dict['detection_boxes'],
               self.output_dict['detection_classes'],
               self.output_dict['detection_scores'],
               self.category_index,
               instance_masks=self.output_dict.get('detection_masks_reframed', None),
               use_normalized_coordinates=True,
               line_thickness=8)
         return self.image_np, self.classed
     
       except:
             self.image_np = vis_util.visualize_boxes_and_labels_on_image_array(
               self.image_np,
               self.output_dict['detection_boxes'],
               self.output_dict['detection_classes'],
               self.output_dict['detection_scores'],
               self.category_index,
               instance_masks= self.output_dict.get('detection_masks_reframed', None),
               use_normalized_coordinates=True,
               line_thickness=8)
             return self.image_np


In [65]:
tf1 = Tensorflow_mobile_SSD()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [68]:

import cv2

class VideoCamera:
    def __init__(self):
        self.vid = cv2.VideoCapture(0)
        self.vid.set(cv2.CAP_PROP_FRAME_WIDTH, 10000)
        self.vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 18000)
        self.vid.set(cv2.CAP_PROP_FPS, 25)
        while True:
            self.ret, self.frame = self.vid.read()
            if self.ret:
                try :
                    self.frame, cls = tf1.show_inference(tf1.detection_model, self.frame)
                except:
                    self.frame = tf1.show_inference(tf1.detection_model, self.frame)
    
                cv2.imshow("fds", self.frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        self.vid.release()
        # Destroy all the windows
        cv2.destroyAllWindows()

        



    def frame_capture(self):
        pass

In [69]:
v2 = VideoCamera()

In [82]:
import time
time.localtime()

time.struct_time(tm_year=2021, tm_mon=3, tm_mday=27, tm_hour=0, tm_min=56, tm_sec=22, tm_wday=5, tm_yday=86, tm_isdst=0)

In [83]:
time.localtime()[4],time.localtime()[5] 

(56, 35)

In [92]:
22, 22+30, 52%60

(22, 52, 52)

In [91]:
60, 60+30, 90%60

(60, 90, 30)